**Importation des bibliothèques nécessaires**


In [116]:
import sqlite3
import pandas as pd
from unidecode import unidecode
from numpy import NaN

# Importation des données CSV dans la base de données `Emploi.db`


## Nettoyage des données


Nous avons d'abord fait correspondre autant que possible les valeurs présentes dans les colonnes. Pour les colonnes `NOMBE**`, nous avons d'abord supprimé tous les caractères accentués (en remplaçant `é` par `e` par exemple) puis en mettant toutes les caractères en majuscule. Ensuite, en comparant les valeurs ayant le même code `BE**`, nous avons modifié manuellement les valeurs pour essayer de les faire correspondre au plus à celles de `NOMBE22`. Nous avons également supprimé les lignes n'ayant aucune valeur à la fois dans la colonne `met`, `xmet` et `smet`.


**Fonctions auxiliaires**


In [117]:
# Fonction pour convertir une colonne en int avec 0 pour valeur non numerique
def convert_to_int_with_0(df, cols):
    for col in cols:
        if df[col].dtypes == "object":
            df[col] = df[col].str.replace(
                ' \t-     ', '0', regex=False).str.replace('*', '0', regex=False).astype("int32")
    return df

**Création des dataframe**


In [118]:
list_df = ["17", "18", "19", "20", "21", "22"]

dfs = {}

for df_str in list_df:
    with open(f'../data/Emplois_20{df_str}.csv', 'r', encoding='utf-8') as csvfile:
        df = pd.read_csv(csvfile, delimiter=';')

    df = convert_to_int_with_0(df, ["met", "xmet", "smet"])
    # Suppression des met vides, puisque si met vide alors xmet et smet vides
    df = df[~(df["met"].isna())]

    # Rennomage des colonnes
    df.rename(inplace=True, columns={"code métier BMO ": "Code métier BMO", "nom_metier BMO": "Nom métier BMO",
              "metier": "Code métier BMO", "nommetier": "Nom métier BMO", "NOM_REG18": "NOM_REG"})

    # Suprresion des lignes avec des code région,dept,bassin vide
    df = df[~((df["NOM_REG"].isna()) & (df["NomDept"].isna())
              & (df[f"BE{df_str}"].isna()))]

    dfs[df_str] = df

Pour les bassins d'emplois (`BE**`) non renseignées on renseigne un code unique généré à partir des noms de régions et de départements, puis conversion de la colonne `BE**` en `int32`.


In [119]:
liste_code_bassin = [101, 102, 103, 104, 105, 1107, 1108, 1109, 1115, 1116, 1117, 1118, 1119, 1124, 1125, 1126, 1127, 1128, 1129, 1137, 1138, 1139, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 201, 202, 203, 204, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 2718, 2719, 2720, 2721, 2726, 2727, 2728, 2729, 2730, 2731, 2733, 2734, 2735, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810, 2811, 2812, 2813, 2814, 2816, 2817, 2818, 2819, 2820, 2821, 2822, 2823, 2824, 2825, 2826, 2827, 2828, 301, 302, 303, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3210, 3211, 3212, 3213, 3214, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 401, 402, 403, 404, 405, 4402, 4404, 4407, 4408, 4411, 4412, 4413, 4414, 4415, 4416, 4417, 4418, 4419, 4420, 4421, 4422, 4423, 4424, 4425, 4426, 4427, 4428, 4429, 4430, 4431, 4432, 4433, 4434, 4435, 4436, 4437, 4438, 4439, 4440, 4441, 4442, 4444, 4446, 4447, 4448, 4449, 4450, 4451, 5203, 5205, 5206, 5208, 5209, 5210, 5211, 5212, 5213, 5214, 5215, 5216, 5217,
                     5218, 5219, 5220, 5221, 5222, 5223, 5224, 5225, 5226, 5227, 5228, 5229, 5230, 5301, 5303, 5304, 5306, 5307, 5308, 5309, 5310, 5311, 5312, 5313, 5314, 5315, 5316, 5317, 5318, 5319, 5320, 5321, 5322, 5323, 5324, 5325, 5326, 5327, 5328, 601, 7501, 7502, 7503, 7504, 7506, 7508, 7509, 7510, 7511, 7512, 7513, 7514, 7515, 7518, 7519, 7520, 7521, 7522, 7523, 7524, 7525, 7526, 7527, 7528, 7529, 7530, 7531, 7532, 7533, 7534, 7535, 7536, 7537, 7538, 7539, 7540, 7541, 7542, 7543, 7544, 7545, 7546, 7547, 7548, 7549, 7601, 7604, 7605, 7606, 7607, 7608, 7610, 7611, 7612, 7613, 7614, 7615, 7617, 7618, 7619, 7620, 7621, 7622, 7623, 7625, 7626, 7627, 7628, 7629, 7630, 7631, 7632, 7633, 7634, 7635, 7636, 7637, 7638, 7639, 7640, 7641, 7642, 7643, 7644, 7645, 7646, 7647, 7648, 7649, 8401, 8402, 8403, 8404, 8405, 8406, 8407, 8408, 8409, 8410, 8411, 8412, 8413, 8414, 8415, 8416, 8417, 8419, 8420, 8421, 8426, 8427, 8428, 8430, 8431, 8432, 8435, 8436, 8437, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8448, 8449, 8450, 8451, 8453, 8454, 8455, 8456, 8457, 8458, 8459, 8460, 8461, 9301, 9302, 9304, 9305, 9306, 9307, 9308, 9311, 9313, 9314, 9315, 9316, 9326, 9329, 9331, 9332, 9333, 9399, 9407, 9408, 9409, 9410, 9411]
dict_code_gen = {}


def gen_code_bassin(reg, dept):
    return int("".join(str(ord(char)) for char in str(reg) + str(dept))) % 10000


def get_code_bassin(reg, dept):
    global liste_code_bassin
    global dict_code_gen

    if (reg, dept) in dict_code_gen.keys():
        return dict_code_gen[(reg, dept)]

    g = gen_code_bassin(reg, dept)

    while (g in liste_code_bassin or g in dict_code_gen.values()):
        g += 1

    liste_code_bassin.append(g)
    dict_code_gen[(reg, dept)] = g

    return g

In [120]:
for df_str in list_df:
    df = dfs[df_str]

    if df.dtypes[f"BE{df_str}"] == 'float64':
        df[f"BE{df_str}"] = df.apply(lambda x: get_code_bassin(x['NOM_REG'], x['NomDept']) if x[f"BE{df_str}"]
                                     != x[f"BE{df_str}"] or x[f"BE{df_str}"] == None else x[f"BE{df_str}"], axis=1)
    df[f"BE{df_str}"] = df[f"BE{df_str}"].astype("int32", errors="raise")

Pour les noms de bassins d'emplois (`NOMBE**`) non renseignées on renseigne un nom généré à partir des noms de régions et de départements, puis suppression des accents et mise en majuscule.


In [121]:
for df_str in list_df:
    df = dfs[df_str]

    df[f"NOMBE{df_str}"] = df.apply(lambda x: f"PAS DE NOM : {x['NomDept']}, {x['NOM_REG']}" if x[f"NOMBE{df_str}"]
                                    != x[f"NOMBE{df_str}"] or x[f"NOMBE{df_str}"] == "" else x[f"NOMBE{df_str}"], axis=1)
    dfs[df_str][f"NOMBE{df_str}"] = dfs[df_str][f"NOMBE{df_str}"].apply(
        unidecode).apply(lambda x: x.upper())

Pour les libellés départements et les régions, on prend directement les valeurs dans les données de 2022.


In [122]:
reg_dict = dfs["22"][["REG", "NOM_REG"]].drop_duplicates().sort_values(
    by="REG").set_index('REG').to_dict()["NOM_REG"]
dept_dict = dfs["22"][["Dept", "NomDept"]].drop_duplicates().sort_values(
    by="Dept").set_index('Dept').to_dict()["NomDept"]


def region(x):
    global reg_dict
    if x["REG"] in reg_dict.keys():
        return reg_dict[x["REG"]]
    reg_dict[x["REG"]] = x["NOM_REG"]
    return x["NOM_REG"]


def dept(x):
    global dept_dict
    dep = '{0:0>2}'.format(x["Dept"])

    if dep in dept_dict.keys():
        return dept_dict[dep]

    dept_dict[dep] = x["NomDept"]

    return x["NomDept"]

In [123]:
for df_str in list_df[::-1]:
    df = dfs[df_str]
    df["NOM_REG"] = df.apply(lambda x: region(x), axis=1)
    df["NomDept"] = df.apply(lambda x: dept(x), axis=1)

### Corrections manuelles apportées


Des modifications manuelles ont été apportées pour corriger des erreurs de données identifiées dans le fichier. Par exemple, en 2019, le code `102` était utilisé pour deux bassins différents. Pour corriger cette erreur, nous avons modifié manuellement le code `102` correspondant au bassin "BASSIN ILES DU NORD" en le remplaçant par le code `104`, qui correspond bien à ce bassin en 2022. De même, pour uniformiser les noms de bassins d'emploi, nous avons modifié manuellement le nom "Saint Yriex" en "ST YRIEX". La majorité des erreurs étaient des changements de noms entre les années. Nous avons donc simplement pris le nom de 2022 s'il correspondait au bassin d'emploi. Les changements effectués sont les suivants :

- Code `102` apparait pour deux bassins différents en **2019** -> Changement de code `102` en `104` pour les NOMBE21 == "BASSIN ILES DU NORD"
- Code `7548` apparait pour deux bassins différents en **2019** (ST YRIEX et SAINT YRIEX) -> "SAINT YRIEX" deviens "ST YRIEX"
- Code `7649` identique avec nom différents entre **2019** et **2022** : `Agde-Pézenas` -> `AGDE-PEZENAS`
- Code `104` identique avec nom différents entre **2019** et **2022** : `BASSIN SAINT-MARTIN` -> `BASSIN ILES DU NORD`
- Code `2808` identique avec nom différents entre **2019** et **2022** : `BOLBEC` -> `LILLEBONNE`
- Code `7520` identique avec nom différents entre **2019** et **2022** : `SAINT JEAN D'ANGELY` -> `SAINT-JEAN-D'ANGELY`
- Code `1153` identique avec nom différents entre **2018** et **2019** : `T25` -> `BRIE-CRECOIS`
- Code `7520` identique avec nom différents entre **2018** et **2019** : `SAINT JEAN D'ANGELY` -> `SAINT-JEAN-D'ANGELY`
- Code `104` identique avec nom différents entre **2017** et **2018** : `SAINT-MARTIN` -> `ASSIN ILES DU NORD`
- Code `1129` identique avec nom différents entre **2017** et **2018** : `T12` -> `T12 GRAND-ORLY SEINE BIEVRE`
- Code `1137` identique avec nom différents entre **2017** et **2018** : `T2` -> `T2 VALLEE SUD GRAND PARIS`
- Code `1138` identique avec nom différents entre **2017** et **2018** : `T5` -> `T5 BOUCLE NORD DE SEINE`
- Code `1139` identique avec nom différents entre **2017** et **2018** : `T3` -> `T3 GRAND PARIS SEINE OUEST`
- Code `1146` identique avec nom différents entre **2017** et **2018** : `T4`-> `T4 PARIS OUEST LA DEFENSE`
- Code `1147` identique avec nom différents entre **2017** et **2018** : `T6` -> `T6 PLAINE COMMUNE`
- Code `1148` identique avec nom différents entre **2017** et **2018** : `T8` -> `T8 EST ENSEMBLE`
- Code `1149` identique avec nom différents entre **2017** et **2018** : `T9` -> `T9 GRAND PARIS GRAND EST`
- Code `1150` identique avec nom différents entre **2017** et **2018** : `T11` -> `T11 GRAND PARIS SUD EST AVENIR`
- Code `1151` identique avec nom différents entre **2017** et **2018** : `T10` -> `T10 PARIS EST MARNE ET BOIS`
- Code `5313` identique avec nom différents entre **2017** et **2018** : `REDON (PARTIEL)` -> `REDON`
- Code `7520` identique avec nom différents entre **2017** et **2018** : `SAINT JEAN D'ANGELY` -> `SAINT-JEAN-D'ANGELY`
- Code `7528` identique avec nom différents entre **2017** et **2018** : `PAUILLAC` -> `LESPARRE-MEDOC`
- Code `9304` identique avec nom différents entre **2018** et **2022** : `BASSIN NICOIS BIS` -> `NICE`
- Code `9305` identique avec nom différents entre **2018** et **2022** : `BASSIN CANNOIS BIS` -> `CANNES`
- Code `9311` identique avec nom différents entre **2018** et **2022** : `BASSIN TOULONNAIS` -> `TOULON`
- Code `9313` identique avec nom différents entre **2018** et **2022** : `BASSIN D'AVIGNON` -> `AVIGNON`
- Code `9326` identique avec nom différents entre **2018** et **2022** : `BASSIN DE LA CASA` -> `LA CASA`
- Code `9331` identique avec nom différents entre **2018** et **2022** : `BASSIN EST VAROIS` -> `EST VAR`
- Code `9332` identique avec nom différents entre **2018** et **2022** : `BASSIN NORD OUEST VAR` -> `HAUT VAR`

Pour les données de 2017, nous avons remarqué que seulement 131 codes basisn correspondaient avec les données des années suivante (donc près 300 codes n'ont pas de correspondance). Nous avons alors décidé que nous n'utiliserons pas les données codes bassin pour cette année. Pour tout de même préserver ces données dans notre BD tout en les différenciants, nous avons décidé de rajouter "17" devant chaque code bassin. 


### Modifications manuelles dans le dataframe 2017 (df_2017)


**Fonctions auxiliaires**


In [124]:
# Fonction pour concatener 17 devant un entier
def concat_17(ent):
    return int("17"+str(ent))

In [125]:
df_2017 = dfs["17"]
df_2017["BE17"] = df_2017["BE17"].apply(concat_17)

### Modifications manuelles dans le dataframe 2018 (df_2018)


In [126]:
df_2018 = dfs["18"]


df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "SAINT-MARTIN", "BASSIN ILES DU NORD",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T12", "T12 GRAND-ORLY SEINE BIEVRE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T2", "T2 VALLEE SUD GRAND PARIS",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T5", "T5 BOUCLE NORD DE SEINE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T3", "T3 GRAND PARIS SEINE OUEST",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T4", "T4 PARIS OUEST LA DEFENSE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace("T6", "T6 PLAINE COMMUNE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace("T8", "T8 EST ENSEMBLE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T9", "T9 GRAND PARIS GRAND EST",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T11", "T11 GRAND PARIS SUD EST",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "T10", "T10 PARIS EST MARNE ET BOIS",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace("REDON (PARTIEL)", "REDON",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "SAINT JEAN D'ANGELY", "SAINT-JEAN-D'ANGELY",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "PAUILLAC", "LESPARRE-MEDOC",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN NICOIS BIS", "NICE",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN CANNOIS BIS", "CANNES",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN TOULONNAIS", "TOULON",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN D'AVIGNON", "AVIGNON",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN DE LA CASA", "LA CASA",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN EST VAROIS", "EST VAR",regex=False)
df_2018["NOMBE18"] = df_2018["NOMBE18"].str.replace(
    "BASSIN NORD OUEST VAR", "HAUT VAR",regex=False)

### Modifications manuelles dans le dataframe 2019 (df_2019)


In [127]:
df_2019 = dfs["19"]

df_2019["NOMBE19"] = df_2019["NOMBE19"].str.replace("T25", "BRIE-CRECOIS",regex=False)
df_2019["NOMBE19"] = df_2019["NOMBE19"].str.replace(
    "SAINT JEAN D'ANGELY", "SAINT-JEAN-D'ANGELY",regex=False)
df_2019["NOMBE19"] = df_2019["NOMBE19"].str.replace(
    "SAINT YRIEIX", "ST YRIEIX",regex=False)

### Modifications manuelles dans le dataframe 2020 (df_2020)


In [128]:
df_2020 = dfs["20"]

df_2020["NOMBE20"] = df_2020["NOMBE20"].str.replace(
    "BASSIN SAINT-MARTIN", "BASSIN ILES DU NORD",regex=False)
df_2020["NOMBE20"] = df_2020["NOMBE20"].str.replace("BOLBEC", "LILLEBONNE",regex=False)
df_2020["NOMBE20"] = df_2020["NOMBE20"].str.replace(
    "SAINT JEAN D'ANGELY", "SAINT-JEAN-D'ANGELY",regex=False)

### Modifications manuelles dans le dataframe 2021 (df_2021)


In [129]:
df_2021 = dfs["21"]

df_2021["NOMBE21"] = df_2021["NOMBE21"].str.replace(
    "Agde-P麥nas", "AGDE-PEZENAS",regex=False)

df_2021["BE21"] = df_2021.apply(
    lambda x: 104 if x["NOMBE21"] == "BASSIN ILES DU NORD" else x["BE21"], axis=1)

In [130]:
for df_str in list_df:
    df_unique = dfs[df_str][[f"BE{df_str}",f"NOMBE{df_str}"]].drop_duplicates()
    for id in df_unique[f"BE{df_str}"]:
        if len(df_unique[df_unique[f"BE{df_str}"]==id])!=1:
            print(df_unique[df_unique[f"BE{df_str}"]==id])

## Importation dans la BD


**Fonctions pour insertion dans la Bd**


In [131]:
def to_db(row, db, df_str):

    alim_familles_metier(row, db, df_str)
    alim_metier(row, db)
    alim_geo(row, db, df_str)
    alim_recrutements(row, db, df_str)


def alim_metier(row, db):

    code_metier_BMO = row["Code métier BMO"]

    nom_metier = row["Nom métier BMO"]
    code_famille_BMO = row["Famille_met"]

    try:
        cur = db.cursor()
        cur.execute("INSERT INTO metiers('code_metier_BMO','nom_metier','code_famille_BMO') VALUES (?,?,?)",
                    (code_metier_BMO, nom_metier, code_famille_BMO))
    except sqlite3.IntegrityError:
        pass
    except Exception as e:
        print("AUTRE ERREUR : ", e)


def alim_familles_metier(row, db, df_str):
    code_famille_BMO = row["Famille_met"]
    nom_famille_BMO = row["Lbl_fam_met"]

    try:
        cur = db.cursor()
        cur.execute("INSERT INTO familles_metier('code_famille_BMO','famille_metier') VALUES (?,?)",
                    (code_famille_BMO, nom_famille_BMO))

    except sqlite3.IntegrityError:
        pass
    except Exception as e:
        print("AUTRE ERREUR : ", e)


def alim_geo(row, db, df_str):
    code_bassin = row[f"BE{df_str}"]
    nom_bassin = row[f"NOMBE{df_str}"]
    dept = row["Dept"]
    nom_dept = row["NomDept"]
    reg = row["REG"]
    nom_reg = row["NOM_REG"]

    try:
        cur = db.cursor()
        cur.execute("INSERT INTO geo('code_bassin','nom_bassin','dept','nom_dept','reg','nom_reg') VALUES (?,?,?,?,?,?)",
                    (code_bassin, nom_bassin, dept, nom_dept, reg, nom_reg))

    except sqlite3.IntegrityError:
        pass
    except Exception as e:
        print("AUTRE ERREUR : ", e)


def alim_recrutements(row, db, df_str):
    met = row["met"]
    xmet = row["xmet"]
    smet = row["smet"]
    code_metier_BMO = row["Code métier BMO"]
    code_bassin = row[f"BE{df_str}"]
    annee = row["annee"]

    try:
        cur = db.cursor()
        cur.execute("INSERT INTO recrutements('met','xmet','smet','code_metier_BMO','code_bassin','annee') VALUES (?,?,?,?,?,?)",
                    (met, xmet, smet, code_metier_BMO, code_bassin, annee))

    except sqlite3.IntegrityError:
        pass
    except Exception as e:
        print("AUTRE ERREUR : ", e)

**Connexion**


In [132]:
db = sqlite3.connect("../Emploi.db")

**Création de la BD**


In [133]:
cur = db.cursor()

# Suppression des anciennes tables
cur.execute("PRAGMA writable_schema = 1")
cur.execute(
    "delete from sqlite_master where type in ('table', 'index', 'trigger')")
cur.execute("PRAGMA writable_schema = 0")
db.commit()

cur.execute("VACUUM")
db.commit()

cur.execute("PRAGMA INTEGRITY_CHECK")
db.commit()

#Création des nouvelles tables
cur.execute("""
CREATE TABLE metiers (
	"code_metier_BMO"	VARCHAR(15) NOT NULL UNIQUE,
	"nom_metier"	VARCHAR(50),
	"code_famille_BMO"	INTEGER NOT NULL,
	PRIMARY KEY("code_metier_BMO"),
	FOREIGN KEY("code_famille_BMO") REFERENCES "familles_metier"("code_famille_BMO"))""")

cur.execute("""
CREATE TABLE familles_metier (
	"code_famille_BMO"	VARCHAR(15) NOT NULL UNIQUE,
	"famille_metier"	VARCHAR(50) NOT NULL,
	PRIMARY KEY("code_famille_BMO"))""")

cur.execute("""
CREATE TABLE recrutements (
	"id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	"met"	INTEGER,
	"xmet"	INTEGER,
	"smet"	INTEGER,
	"code_metier_BMO"	VARCHAR(50) NOT NULL,
	"code_bassin"	VARCHAR(50) NOT NULL,
	"annee"	INTEGER NOT NULL,
	FOREIGN KEY("code_metier_BMO") REFERENCES "metier"("code_metier_BMO"),
	FOREIGN KEY("code_bassin") REFERENCES "geo"("code_bassin")
)""")

cur.execute("""
CREATE TABLE geo (
	"code_bassin"	VARCHAR(50) NOT NULL UNIQUE,
	"nom_bassin"	VARCHAR(50),
	"dept"	INTEGER,
	"nom_dept"	VARCHAR(50),
	"reg"	INTEGER,
	"nom_reg"	VARCHAR(50),
	PRIMARY KEY("code_bassin")
)""")


db.commit()

**Insertion dans la BD**


In [134]:
for df_str in list_df:
    for index, row in dfs[df_str].iterrows():
        to_db(row, db, df_str)

db.commit()
db.close()